# Flux1

The CoreAI container is started using the instructions described in the shared `README.md` file which uses `podman` or `docker`.

To enhance the functionality of the CoreAI  environment, we need to install some libraries not pre-installed but required for this notebook. 

## Pre-requisites
Open your terminal or command prompt within the Jupyter notebook. Navigate via `File -> New -> Terminal`.
Type `bash` to access a shell compatible with the following commands.
Navigate to the project directory where you want to set up the environment:

```bash
export PROJECT_NAME="Flux1"
export PIP_CACHE_DIR=`pwd`/.cache/pip
mkdir -p $PIP_CACHE_DIR
python -m venv --system-site-packages myvenv
source myvenv/bin/activate
pip install ipykernel
python -m ipykernel install --user --name=${PROJECT_NAME}-myvenv --display-name="Python (${PROJECT_NAME}-myvenv)"
echo ""; echo "Before continuing load the created Python kernel: Python (${PROJECT_NAME}-myvenv)"
```

Load the Python kernel described above before running the cell below (it might take a few seconds for the kernel to appear in the list of kernels).


## Install Required Libraries:

Before running the following commands, load the `Python (myvenv)` kernel.

Ensure you are in the directory where the Jupyter Notebook and the `myvenv` directory are located. 

In [ ]:
!. ./myvenv/bin/activate; pip install -r requirements.txt

Add `accelerate` to the PATH

In [ ]:
import os
pwd = os.getcwd()
os.environ['PATH'] =  os.path.join(pwd, 'myvenv/bin') + os.pathsep + os.environ['PATH']

! echo $PATH
! which accelerate

**Make sure `HF_HOME` is set BEFORE `transformers` is loaded**

In [ ]:
import os
os.makedirs('HF_HOME', exist_ok=True)
os.environ['HF_HOME'] = 'HF_HOME'

Verify that the `HF_TOKEN` is set before attempting to load the model (this is passed as an environment variable within the `start_CoreAI.sh` script.

In [ ]:
# Load environment configuration file
# This sets up the basic structure for API credentials

import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if 'HF_TOKEN' not in os.environ:
    printf("No HF_TOKEN set, will not be able to download the model")
    exit(1)

hf_token=os.environ['HF_TOKEN']

## Running Flux1

### Obtain model

In [ ]:
import torch
from diffusers import FluxPipeline

In [ ]:
# Free up some memory before testing
import gc

# check memory
print(torch.cuda.memory_allocated())

gc.collect()
torch.cuda.empty_cache()

# check memory again
print(torch.cuda.memory_allocated())

In [ ]:
pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-dev")
pipe.enable_sequential_cpu_offload()
torch.cuda.empty_cache()

### Image generation

Modify the `prompt`, `seed` and `width` and `height` as required.

In [ ]:
import random
from IPython.display import Image, display

prompts = [
    "An ocean reflection on starry night with the word JetStream in the sky with whales and dolpins in the ocean",
    "An galaxy with the word OpenStack spelled with stars",
    "A friendly robot with wheels on a mars landscape moving toward the setting sun with the word Exosphere rising from the horizon",
    "The sentence \"Happy Birthday OpenStack and Scientific SIG\" on a marvelous tropical forest"
]

width = 1360
height = 768

c = 1
while c < 5:

    d = 1
    for prompt in prompts:
        seed = random.randint(1, 999)
    
        image = pipe(
            prompt,
            guidance_scale=0.0,
            num_inference_steps=4,
            max_sequence_length=256,
            height=height,
            width=width,
            generator=torch.Generator("cpu").manual_seed(seed)
        ).images[0]
        fname = f"flux-schnell-{c}-{d}.png"
        image.save(fname)

        print(f"Prompt: {prompt}")
        display(Image(filename=fname))
        d += 1
    c += 1